In [6]:
# File: 02_get_contact_and_OA_info
# Takes file (name) from file 01_get_article_data and retrieves contact information (names, titles, department, email) for MU
#      authors (currently, corresponding authors with MU affiliations).

import json, re
import pandas as pd

#articleData = pd.read_csv('data/2019-ALL-scopus_plus_unpaywall.csv', encoding='utf-8')
articleData = pd.read_csv('', encoding='utf-8') #filename is first argument

In [ ]:
from ldap3 import Server, Connection, ALL, NTLM
server_address = '' #LDAP/directory server address here
server = Server(server_address, get_info=ALL)
conn = Connection(server, user="", password="", authentication=NTLM, auto_bind=True) #not ideal, but for now is coded that you have to put your username/password here; CAREFUL with Github (should put this in a config file)
print(conn)
conn.extend.standard.who_am_i()

In [ ]:
articleData.keys()

In [ ]:
# Look for corresponding authors @missouri.edu
# TRY TO ADD CONDITIONAL FOR @health (added 10/12/2020)
mu_corr_emails = []
mu_corr_givenNames = []
mu_corr_surnames = []
mu_corr_departments = []

directory1 = '' #domain definition goes here, ie 'dc=umsystem, dc=edu'
directory2 = '' #domain definition for health sciences (or other secondary directory)


#For each corresponding author, see if they have an @missouri.edu email address, and if so, collect their information (your directory configuration may well be different, available attributes or their names, etc.)
#Because we're going to match up these rows with the original article data rows, if there are no matches, or no correspondence address at all (all of the "else" conditions), append '' to the column
for index, row in articleData.iterrows():
#    print(row['Correspondence Address'])
    if (not pd.isnull(row['Correspondence Address'])):
        match = re.search(r'[\w\.-]+@[\w\.]*missouri\.edu', str(row['Correspondence Address']))
        if(match is not None):
            print(match.group())
            mu_corr_emails.append(match.group())
            conn.search(directory1, '(mail='+match.group()+')', attributes=['mail','title', 'sn', 'initials', 'middleName', 'givenName', 'department'])
            print(len(conn.entries))
            if len(conn.entries) == 1:
                print(conn.entries[0].givenName, conn.entries[0].initials, conn.entries[0].sn, conn.entries[0].department)
                mu_corr_givenNames.append(conn.entries[0].givenName)
                mu_corr_surnames.append(conn.entries[0].sn)
                mu_corr_departments.append(conn.entries[0].department)
            else: #try @health
                conn.search(directory2, '(mail='+match.group()+')', attributes=['mail','title', 'sn', 'initials', 'middleName', 'givenName', 'department'])
                print(len(conn.entries))
                if len(conn.entries) == 1:
                    print(conn.entries[0].givenName, conn.entries[0].initials, conn.entries[0].sn, conn.entries[0].department)
                    mu_corr_givenNames.append(conn.entries[0].givenName)
                    mu_corr_surnames.append(conn.entries[0].sn)
                    mu_corr_departments.append(conn.entries[0].department)
                else:
                    mu_corr_givenNames.append('')
                    mu_corr_surnames.append('')
                    mu_corr_departments.append('')
        else:
            mu_corr_emails.append('')
            mu_corr_givenNames.append('')
            mu_corr_surnames.append('')
            mu_corr_departments.append('')

    else:
        mu_corr_emails.append('')
        mu_corr_givenNames.append('')
        mu_corr_surnames.append('')
        mu_corr_departments.append('')

In [ ]:
#Add the name arrays as new columns to our articleData dataframe
articleData['corr email'] = mu_corr_emails
articleData['corr given name'] = mu_corr_givenNames
articleData['corr surname'] = mu_corr_surnames
articleData['corr department'] = mu_corr_departments

In [ ]:
#Save to .csv (add filename)
articleData.to_csv('', encoding='utf-8-sig') #utf-8-sig adds the BOM character, which seems to help Excel properly read the encoding of special character, if the file is opened in Excel